# Plot HRRR Statistics from OSG Calculation

In [65]:
%matplotlib inline
import h5py
import matplotlib.pyplot as plt
import multiprocessing
import numpy as np
from collections import OrderedDict

import sys
import sys
sys.path.append('/uufs/chpc.utah.edu/common/home/u0553130/pyBKB_v2')
from BB_downloads.HRRR_S3 import *
from BB_basemap.draw_maps import draw_CONUS_HRRR_map
from BB_MesoWest.MesoWest_STNinfo import get_station_info
from BB_data.grid_manager import pluck_point_new

In [66]:
from matplotlib.dates import DateFormatter
formatter = DateFormatter('%b\n%d')

import matplotlib as mpl 
mpl.rcParams['figure.figsize'] = [10, 8]
mpl.rcParams['figure.titlesize'] = 15
mpl.rcParams['figure.titleweight'] = 'bold'
mpl.rcParams['xtick.labelsize'] = 10
mpl.rcParams['ytick.labelsize'] = 10
mpl.rcParams['axes.labelsize'] = 10
mpl.rcParams['axes.titlesize'] = 12
mpl.rcParams['lines.linewidth'] = 1.8
mpl.rcParams['grid.linewidth'] = .25
mpl.rcParams['figure.subplot.wspace'] = 0.05
mpl.rcParams['figure.subplot.hspace'] = 0.05
mpl.rcParams['legend.fontsize'] = 8
mpl.rcParams['legend.framealpha'] = .75
mpl.rcParams['legend.loc'] = 'best'
mpl.rcParams['savefig.bbox'] = 'tight'
mpl.rcParams['savefig.dpi'] = 100

In [67]:
m = draw_CONUS_HRRR_map()

### Get lat/lon grid from file for plotting

In [68]:
DIR = '/uufs/chpc.utah.edu/common/home/horel-group2/blaylock/HRRR_OSG/'
latlon_file = h5py.File(DIR+'OSG_HRRR_latlon.h5', 'r')
lat = latlon_file['latitude'].value
lon = latlon_file['longitude'].value

### Plot the OSG Statistics Map

In [69]:
def plot_OSG_map(args):
    month, day, hour, fxx, statistic, var = args

    variable = var.replace(":", '_').replace(' ', '_')

    # open a file
    FILE = 'OSG_HRRR_%s_m%02d_d%02d_h%02d_f%02d.h5' % (variable, month, day, hour, fxx)
    
    if os.path.exists(DIR+FILE):
        h = h5py.File(DIR+FILE, 'r')
    else:
        print "exited", DIR+FILE
        return None

    cores = h['cores'].value
    count = h['count'].value
    
    sDATE = h['Beginning Date'].value
    eDATE = h['Ending Date'].value
    timer = h['timer'].value
      
    # A list of the available percentiles in the 'percentile value' key
    percentiles = h['percentile'].value

    STAT = h[statistic].value
    m.drawstates()
    m.drawcoastlines()
    m.drawcountries()
    if var == 'TMP:2 m':
        m.pcolormesh(lon, lat, STAT-273.15,
                     vmin=-8, vmax=38,
                     cmap='Spectral_r', latlon=True)
        cb = plt.colorbar(orientation='horizontal', pad=0.01, shrink=.95, extend="both")
        cb.set_label('2 m Temperature (C)')
    if var == 'DPT:2 m':
        m.pcolormesh(lon, lat, STAT-273.15,
                     vmin=-10, vmax=25,
                     cmap='BrBG', latlon=True)
        cb = plt.colorbar(orientation='horizontal', pad=0.01, shrink=.95, extend="both")
        cb.set_label('2 m Dew Point (C)')
    elif var == 'WIND:10 m':
        m.pcolormesh(lon, lat, STAT, vmin=0,
                     vmin=0, vmax=20,
                     cmap='plasma_r', latlon=True)
        cb = plt.colorbar(orientation='horizontal', pad=0.01, shrink=.95, extend="max")
        cb.set_label(r'10 m Wind Speed (ms$\mathregular{^{-1}}$)')
    elif var == 'REFC:entire':
        # Mask out empty reflectivity values
        dBZ = STAT
        dBZ = np.ma.array(dBZ)
        dBZ[dBZ == -10] = np.ma.masked
        m.pcolormesh(lon, lat, dBZ,
                     vmax=0, vmin=80,
                     cmap='gist_ncar', latlon=True)
        cb = plt.colorbar(orientation='horizontal', pad=0.01, shrink=.95,)
        cb.set_label(r'Simulated Composite Reflectivity (dBZ)')
    
    months = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC',]
    plt.title('HRRR Composite: %s \n %s-%02d Hour:%02d, fxx:%02d +/- 15 Days\nFirst: %s, Last:%s\nCount:%02d, Cores:%02d\nOSG Timer: %s' % (statistic, months[month-1], day, hour, fxx, sDATE, eDATE, count, cores, timer))

    SAVEDIR = '/uufs/chpc.utah.edu/common/home/u0553130/public_html/PhD/HRRR/OSG/daily30/%s_%s/' % (variable, statistic)
    if not os.path.exists(SAVEDIR):
        os.makedirs(SAVEDIR)
    plt.savefig(SAVEDIR+'OSG_%s_%s_m%02d_d%02d_h%02d_f%02d.png' % (statistic, variable, month, day, hour, fxx), bbox_inches='tight', dpi=100)
    plt.close()
    h.close()

SyntaxError: keyword argument repeated (<ipython-input-69-b525231d596e>, line 43)

-------

### Input Arguments
statistic can be from the list:  
`mean, p00, p01, p02, p03, p04, p05, p10, p15, p25, p33, p50, p66, p75, p90, p95, p96, p97, p98, p99, p100`

In [70]:
fxx = 0
statistic = 'p95'
var = 'TMP:2 m'

##### Additional Setup

In [71]:
variable = var.replace(':', '_').replace(' ', '_')
DIR = '/uufs/chpc.utah.edu/common/home/horel-group2/blaylock/HRRR_OSG/hourly30/%s/' % (variable)

months = range(1,13)
days = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
hours = range(24)
hours = [18]

# Dates (with leap year)
HOURS = 366*24
DATES = np.array([datetime(2016, 1, 1) + timedelta(hours = h) for h in range(HOURS)])
DATES = DATES[::24/len(hours)] # in case we don't request all 24 hours

## Plot a map of the statistic

In [ ]:
print "Variable:", var
print "Statistic:", statistic

args = [[month, day, hour, fxx, statistic, var] \
        for month in months \
        for day in range(1,days[month-1]+1) \
        for hour in hours]

# Multiprocessing :)
num_proc = multiprocessing.cpu_count() # use all processors
p = multiprocessing.Pool(num_proc)
result = p.map(plot_OSG_map, args)
p.close()

Variable: TMP:2 m
Statistic: p95
